# Harnessing Object and Scene Semantics for Large-Scale Video Understanding

[paper link](http://cs.brown.edu/~ls/Publications/cvpr2016wu.pdf)

## 动机

识别vedio中的目标（object）和场景语义（scene semantics）。

## 模型架构

包含了三种信息流的语义融合网络（semantic fusion network），分为三层：1.基于帧的低层次的CNN特征；2.基于目标（object）的大型CNN，用来识别对象，可识别20K个类；3.场景特征的大型CNN，可以识别205个场景。

使用object/scene 关系对 作为语义的表示（semantic representation）。

![](https://raw.githubusercontent.com/applenob/paper_note/master/res/hoss-model.png)
![](https://raw.githubusercontent.com/applenob/paper_note/master/res/hoss-model-2.png)

## 数学

训练数据：$D$，每段视频的类标是$z_i∈ Z_{Tr}$，$D=\{(x_i,z_i)\}_{i=1,...,n_{Tr}}$，其中，$n_{Tr}$是训练视频的个数；视频：$V_i=\{f_{i,1},...,f_{i,n_i}\}$，其中$n_i$是$V_i$的帧数。

